In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from pathlib import Path

# === Fonctions ===
def business_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1][0]
    fp = cm[0][1]
    return 10 * fn + fp

def evaluate_model(model, X, y, name):
    probas = model.predict_proba(X)[:, 1]
    preds = (probas >= 0.5).astype(int)
    cost = business_score(y, preds)
    auc = roc_auc_score(y, probas)
    return {"model": name, "business_score": cost, "AUC": auc}

# === Chargement des données ===
train = pd.read_csv("../data/processed/train_clean.csv")
test = pd.read_csv("../data/processed/test_clean.csv")



# Suppression des lignes avec NaN dans TARGET (sécurité)
train = train.dropna(subset=["TARGET"])
test = test.dropna(subset=["TARGET"])

X_test = test.drop(columns=["TARGET", "SK_ID_CURR"], errors="ignore")
y_test = test["TARGET"]

# === Chargement des modèles ===
models_dir = Path("../src/models")
results = []

for model_file in models_dir.glob("*.pkl"):
    try:
        model = joblib.load(model_file)
        model_name = model_file.stem
        res = evaluate_model(model, X_test, y_test, model_name)
        results.append(res)
    except Exception as e:
        print(f"⚠️ Erreur avec le modèle {model_file.name} : {e}")

⚠️ Erreur avec le modèle LightGBM_custom.pkl : Found array with 0 sample(s) (shape=(0, 112)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle LightGBM_pipeline.pkl : Found array with 0 sample(s) (shape=(0, 112)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle RandomForest_smote.pkl : Found array with 0 sample(s) (shape=(0, 247)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle XGBoost_custom.pkl : Found array with 0 sample(s) (shape=(0, 112)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle RandomForest_pipeline.pkl : Found array with 0 sample(s) (shape=(0, 112)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle XGBoost_smote.pkl : Found array with 0 sample(s) (shape=(0, 247)) while a minimum of 1 is required by SimpleImputer.
⚠️ Erreur avec le modèle XGBoost_pipeline.pkl : Found array with 0 sample(s) (shape=(0, 112)) while a minimum of 1 is required b

In [11]:
print(results)

[]


In [ ]:
# === Résultats ===
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="AUC", ascending=False)

print("\n📊 Résultats comparatifs :")
print(results_df)



In [ ]:
# === Visualisation ===
plt.figure(figsize=(10, 5))
plt.bar(results_df["model"], results_df["AUC"])
plt.title("AUC de chaque modèle sur le jeu de test")
plt.xticks(rotation=45)
plt.ylabel("AUC")
plt.tight_layout()
plt.show()